# Benchmark Evaluation Notebook

## Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process.

In [1]:
# !pip install uv
# !uv pip install  -r requirements.txt

In [1]:
# import snowflake
# from snowflake.snowpark.context import get_active_session
# session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os

## Input Params

In [25]:
feature_to_test = 'NDTI'
feature_dataset_path = "../development/landsat_features_training_all_bands_with_index.csv"

In [19]:
train_features = None
if feature_to_test is not None:
    train_features = pd.read_csv(feature_dataset_path)
    display(train_features.head(5))

## Response Variable

In [2]:
Water_Quality_df = pd.read_csv("water_quality_training_dataset.csv")
Water_Quality_df["sample_location_group"] = Water_Quality_df.groupby(['Longitude', 'Latitude']).ngroup()
# display(Water_Quality_df.head(5))

In [6]:
landsat_train_features = pd.read_csv("landsat_features_training.csv")
landsat_train_features['NDMI'] = landsat_train_features['NDMI'].astype(float)
landsat_train_features['MNDWI'] = landsat_train_features['MNDWI'].astype(float)
# display(landsat_train_features.head(5))

In [7]:
Terraclimate_df = pd.read_csv("../terraclimate_features_training.csv")
# display(Terraclimate_df.head(5))

In [20]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2,dataset3):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined
    dataset2 - Dataset 2 to be combined
    '''

    data = pd.concat([dataset1,dataset2,dataset3], axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

# Combine two datasets vertically (along columns) using pandas concat function.
def combine_dataset_list(dataset_list):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset_list - List of datasets to be combined in order
    '''

    data = pd.concat(dataset_list, axis=1)
    data = data.loc[:, ~data.columns.duplicated()]
    return data

In [22]:
wq_data = None
if train_features is None:
    # Combining ground data and final data into a single dataset.
    wq_data = combine_two_datasets(Water_Quality_df, landsat_train_features, Terraclimate_df)
    display(wq_data.head(5))
else:
    # Combining ground data and final data into a single dataset.
    wq_data = combine_dataset_list([Water_Quality_df, landsat_train_features, Terraclimate_df, train_features])
    display(wq_data.head(5))

### Handling Missing Values

Before model training, missing values in the dataset were carefully handled to ensure data consistency and prevent model bias. Numerical columns were imputed using their median values, maintaining the overall data distribution while minimizing the impact of outliers.

In [23]:
wq_data = wq_data.fillna(wq_data.median(numeric_only=True))
wq_data.isna().sum()

## Model Building

Now let us select the columns required for our model-building exercise. We will consider only **SWIR22**, **NDMI**, and **MNDWI** from the Landsat data, and **PET** from the TerraClimate dataset as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the water quality parameters.

In [26]:
# Retaining only the columns for swir22, NDMI, MNDWI, pet, Total Alkalinity, Electrical Conductance and Dissolved Reactive Phosphorus Index in the dataset.
if feature_to_test is None:
    wq_data = wq_data[["sample_location_group", 'swir22','NDMI','MNDWI','pet', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']]
else:
    wq_data = wq_data[["sample_location_group", feature_to_test, 'swir22','NDMI','MNDWI','pet', 'Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']]

In [27]:
def split_data(X, y, test_size=0.3, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, scaler

def train_model(X_train_scaled, y_train):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)
    return model

def evaluate_model(model, X_scaled, y_true, dataset_name="Test"):
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # print(f"\n{dataset_name} Evaluation:")
    # print(f"R²: {r2:.3f}")
    # print(f"RMSE: {rmse:.3f}")
    return y_pred, r2, rmse

In [28]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import numpy as np

def run_groupkfold_cv(X, y, groups, n_splits=5, param_name="Parameter"):
    gkf = GroupKFold(n_splits=n_splits)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
        # print(f"\n=== Fold {fold+1} ===")

        # Split
        X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]

        # Scale
        X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)

        # Train
        model = train_model(X_train_scaled, y_train)

        # Evaluate (in-sample)
        y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")

        # Evaluate (out-sample)
        y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")

        fold_results.append((r2_train, rmse_train, r2_test, rmse_test))

    df_results_kfold = pd.DataFrame(fold_results, columns=['R2_Train', 'RMSE_Train', 'R2_Test', 'RMSE_Test']).reset_index().rename(columns={"index": "fold"})
    df_results_kfold['Parameter'] = param_name
    df_results_kfold['Features'] = ', '.join([col for col in X.columns if col != 'sample_location_group'])
    df_results_kfold = df_results_kfold[['Parameter', 'Features', 'R2_Train', 'RMSE_Train', 'R2_Test', 'RMSE_Test']]

    return df_results_kfold

## Model Workflow (Pipeline)

The complete model development process follows a structured pipeline to ensure consistency, reproducibility, and clarity. Each stage in the workflow is modularized into independent functions that can be reused for different water quality parameters. This modular approach streamlines the process and makes the workflow easily adaptable to new datasets or parameters in the future.

The pipeline automates the sequence of steps — from data preparation to evaluation — for each target parameter. The same set of predictor variables is used, while the response variable changes for each of the three targets: *Total Alkalinity (TA)*, *Electrical Conductance (EC)*, and *Dissolved Reactive Phosphorus (DRP)*. By maintaining a consistent framework, comparisons across models remain fair and interpretable.

In [29]:
def run_pipeline(X, y, param_name="Parameter"):
    # print(f"\n{'='*60}")
    # print(f"Training Model for {param_name}")
    # print(f"{'='*60}")
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # Scale
    X_train_scaled, X_test_scaled, scaler = scale_data(X_train, X_test)
    
    # Train
    model = train_model(X_train_scaled, y_train)
    
    # Evaluate (in-sample)
    y_train_pred, r2_train, rmse_train = evaluate_model(model, X_train_scaled, y_train, "Train")
    
    # Evaluate (out-sample)
    y_test_pred, r2_test, rmse_test = evaluate_model(model, X_test_scaled, y_test, "Test")
    
    # Return summary
    results = {
        "Parameter": param_name,
        "Features": ', '.join([col for col in X.columns if col != 'sample_location_group']),
        "R2_Train": r2_train,
        "RMSE_Train": rmse_train,
        "R2_Test": r2_test,
        "RMSE_Test": rmse_test
    }
    return model, scaler, pd.DataFrame([results])

### Model Training and Evaluation for Each Parameter

In this step, we apply the complete modeling pipeline to each of the three selected water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. The input feature set (`X`) remains the same across all three models, while the target variable (`y`) changes for each parameter. 

For every parameter, the `run_pipeline()` function is executed, which handles data preprocessing, model training, and both in-sample and out-of-sample evaluation. This ensures a consistent workflow and allows for a fair comparison of model performance across different water quality indicators.

In [30]:
X = wq_data.drop(columns=['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus'])

y_TA = wq_data['Total Alkalinity']
y_EC = wq_data['Electrical Conductance']
y_DRP = wq_data['Dissolved Reactive Phosphorus']

model_TA, scaler_TA, results_TA = run_pipeline(X, y_TA, "Total Alkalinity")
model_EC, scaler_EC, results_EC = run_pipeline(X, y_EC, "Electrical Conductance")
model_DRP, scaler_DRP, results_DRP = run_pipeline(X, y_DRP, "Dissolved Reactive Phosphorus")

df_results = pd.concat([results_TA, results_EC, results_DRP], axis=0)
display(df_results)

In [31]:
groups = wq_data['sample_location_group']   # or station_id, river_id, etc.
results_TA = run_groupkfold_cv(X, y_TA, groups, param_name="Total Alkalinity")
results_EC = run_groupkfold_cv(X, y_EC, groups, param_name="Electrical Conductance")
results_DRP = run_groupkfold_cv(X, y_DRP, groups, param_name="Dissolved Reactive Phosphorus")

df_results = pd.concat([results_TA, results_EC, results_DRP], axis=0)
display(df_results)

df_result_mean = df_results.groupby(['Parameter', 'Features']).mean()#.rename(columns={'R2': 'R2 (mean)', 'RMSE': 'RMSE (mean)'})
display(df_result_mean.sort_values('Parameter', ascending=False))

### Model Performance Summary

After training and evaluating the models for each water quality parameter, the individual performance metrics are combined into a single summary table. This table consolidates the R² and RMSE values for both in-sample and out-of-sample evaluations, enabling an easy comparison of model performance across Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus. 

Such a summary provides a quick overview of how well each model captures the variability in each parameter and highlights any differences in predictive accuracy.

In [12]:
results_summary = pd.concat([results_TA, results_EC, results_DRP], ignore_index=True)
results_summary

## Submission (NO NOT USE BELOW)

Once you are satisfied with your model’s performance, you can proceed to make predictions for unseen data. To do this, use your trained model to estimate the concentrations of the target water quality parameters — Total Alkalinity, Electrical Conductance, and Dissolved Reactive Phosphorus — for a set of test locations provided in the **Submission_template.csv** file. 

The predicted results can then be uploaded to the challenge platform for evaluation.

In [ ]:
test_file = pd.read_csv("submission_template.csv")
display(test_file.head(5))

In [ ]:
landsat_val_features = pd.read_csv("landsat_features_validation.csv")
display(landsat_val_features.head(5))

In [ ]:
Terraclimate_val_df = pd.read_csv("terraclimate_features_validation.csv")
display(Terraclimate_val_df.head(5))

Similarly, participants can use the **Landsat** and **TerraClimate** data extraction demonstration notebooks to produce feature CSVs for their **validation** data. For convenience, we have already computed and saved example validation outputs as `landsat_features_val_V3.csv` and `Terraclimate_val_df_v3.csv`. 

Participants should save their own extracted files in the same format and column schema; doing so will allow this benchmark notebook to load the validation features directly and run smoothly.

In [16]:
#Consolidate all the extracted bands and features in a single dataframe
val_data = pd.DataFrame({
    'Longitude': landsat_val_features['Longitude'].values,
    'Latitude': landsat_val_features['Latitude'].values,
    'Sample Date': landsat_val_features['Sample Date'].values,
    'nir': landsat_val_features['nir'].values,
    'green': landsat_val_features['green'].values,
    'swir16': landsat_val_features['swir16'].values,
    'swir22': landsat_val_features['swir22'].values,
    'NDMI': landsat_val_features['NDMI'].values,
    'MNDWI': landsat_val_features['MNDWI'].values,
    'pet': Terraclimate_val_df['pet'].values,
})

In [17]:
# Impute the missing values
val_data = val_data.fillna(val_data.median(numeric_only=True))

In [ ]:
# Extracting specific columns (swir22, NDMI, MNDWI, pet) from the validation dataset
submission_val_data=val_data.loc[:,['swir22','NDMI','MNDWI','pet']]
display(submission_val_data.head())

In [19]:
submission_val_data.shape

In [20]:
# --- Predicting for Total Alkalinity ---
X_sub_scaled_TA = scaler_TA.transform(submission_val_data)
pred_TA_submission = model_TA.predict(X_sub_scaled_TA)

# --- Predicting for Electrical Conductance ---
X_sub_scaled_EC = scaler_EC.transform(submission_val_data)
pred_EC_submission = model_EC.predict(X_sub_scaled_EC)

# --- Predicting for Dissolved Reactive Phosphorus ---
X_sub_scaled_DRP = scaler_DRP.transform(submission_val_data)
pred_DRP_submission = model_DRP.predict(X_sub_scaled_DRP)

In [21]:
submission_df = pd.DataFrame({
    'Longitude': test_file['Longitude'].values,
    'Latitude': test_file['Latitude'].values,
    'Sample Date': test_file['Sample Date'].values,
    'Total Alkalinity': pred_TA_submission,
    'Electrical Conductance': pred_EC_submission,
    'Dissolved Reactive Phosphorus': pred_DRP_submission
})

In [22]:
#Displaying the sample submission dataframe
display(submission_df.head())

In [23]:
#Dumping the predictions into a csv file.
submission_df.to_csv("/tmp/submission.csv",index = False)

In [ ]:
session.sql(f"""
    PUT file:///tmp/submission.csv
    snow://workspace/USER$.PUBLIC.DEFAULT$/versions/live/
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

### Upload submission file on platform

Upload the `submission.csv` file on the challenge platform to generate your score on the leaderboard.

## Conclusion

Now that you have learned a basic approach to model training, it’s time to explore your own techniques and ideas! Feel free to modify any of the functions presented in this notebook to experiment with alternative preprocessing steps, feature engineering strategies, or machine learning algorithms. 

We look forward to seeing your enhanced model and the insights you uncover. Best of luck with the challenge!